In [2]:
import subprocess
import json
import re

def update_requirements_from_pip_list(requirements_file="requirements.txt"):
    """
    Updates the versions in requirements.txt based on the output of 'pip list'.
    Only updates packages already listed in the file.
    """
    try:
        # Get pip list output in JSON format
        pip_list_output = subprocess.run(
            ["pip", "list", "--format", "json"], capture_output=True, text=True, check=True
        )
        installed_packages = json.loads(pip_list_output.stdout)

        # Create a dictionary for easy lookup: {package_name: version}
        installed_versions = {pkg["name"].lower(): pkg["version"] for pkg in installed_packages}

        updated_lines = []
        with open(requirements_file, "r") as f:
            for line in f:
                original_line = line.strip()
                if not original_line or original_line.startswith("#"):
                    updated_lines.append(original_line)
                    continue

                # Extract package name (ignoring version specifiers)
                package_match = re.match(r"([a-zA-Z0-9._-]+)", original_line)
                if package_match:
                    package_name = package_match.group(1).lower()  # Convert to lowercase for consistent matching
                    if package_name in installed_versions:
                        installed_version = installed_versions[package_name]
                        updated_lines.append(f"{package_name}=={installed_version}")
                    else:
                        print(f"Warning: {package_name} from requirements.txt not found in 'pip list' output. Keeping original line.")
                        updated_lines.append(original_line)
                else:
                    updated_lines.append(original_line) # Keep lines that don't match the package format

        # Write the updated lines back to the requirements.txt file
        with open(requirements_file, "w") as f:
            for line in updated_lines:
                f.write(line + "\n")
        print(f"Successfully updated {requirements_file} with relevant package versions from 'pip list'.")

    except subprocess.CalledProcessError as e:
        print(f"Error running 'pip list': {e}")
        print(f"Command output: {e.stdout}")
        print(f"Error output: {e.stderr}")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON output from 'pip list': {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    update_requirements_from_pip_list()


Successfully updated requirements.txt with relevant package versions from 'pip list'.
